## Importing Libraries

In [174]:
!pip install pytest-warnings

In [176]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import neattext.functions as nfx
from sklearn.metrics.pairwise import linear_kernel
import warnings
warnings.filterwarnings("ignore")

## Load Dataset for Recommendation

In [177]:
data=pd.read_csv('udemy_course_data.csv')
data.head(2)

course_id                                       course_title  \
0    1070968                 Ultimate Investment Banking Course   
1    1113822  Complete GST Course & Certification - Grow You...   

                                                 url  is_paid  price  \
0  https://www.udemy.com/ultimate-investment-bank...     True    200   
1      https://www.udemy.com/goods-and-services-tax/     True     75   

   num_subscribers  num_reviews  num_lectures       level content_duration  \
0             2147           23            51  All Levels        1.5 hours   
1             2792          923           274  All Levels         39 hours   

    published_timestamp           subject  profit published_date  \
0  2017-01-18T20:58:58Z  Business Finance  429400     2017-01-18   
1  2017-03-09T16:34:20Z  Business Finance  209400     2017-03-09   

  published_time  year  month  day  
0      20:58:58Z  2017      1   18  
1      16:34:20Z  2017      3    9

In [179]:
data.shape

(3683, 18)

In [180]:
data.columns

Index(['course_id', 'course_title', 'url', 'is_paid', 'price',
       'num_subscribers', 'num_reviews', 'num_lectures', 'level',
       'content_duration', 'published_timestamp', 'subject', 'profit',
       'published_date', 'published_time', 'year', 'month', 'day'],
      dtype='object')

In [182]:
data.isnull().sum()

course_id              0
course_title           0
url                    0
is_paid                0
price                  0
num_subscribers        0
num_reviews            0
num_lectures           0
level                  0
content_duration       0
published_timestamp    0
subject                0
profit                 0
published_date         0
published_time         1
year                   0
month                  0
day                    0
dtype: int64

In [183]:
df=data[['course_title','url']] #'price','num_subscribers'
df.head()

course_title  \
0                 Ultimate Investment Banking Course   
1  Complete GST Course & Certification - Grow You...   
2  Financial Modeling for Business Analysts and C...   
3  Beginner to Pro - Financial Analysis in Excel ...   
4       How To Maximize Your Profits Trading Options   

                                                 url  
0  https://www.udemy.com/ultimate-investment-bank...  
1      https://www.udemy.com/goods-and-services-tax/  
2  https://www.udemy.com/financial-modeling-for-b...  
3  https://www.udemy.com/complete-excel-finance-c...  
4  https://www.udemy.com/how-to-maximize-your-pro...

Here  i am using  neattext.functions as nfx for data cleaning , so let first check wht function are present

In [184]:
df.dtypes

course_title    object
url             object
dtype: object

In [185]:
dir(nfx)

['BTC_ADDRESS_REGEX',
 'CURRENCY_REGEX',
 'CURRENCY_SYMB_REGEX',
 'Counter',
 'DATE_REGEX',
 'EMAIL_REGEX',
 'EMOJI_REGEX',
 'HASTAG_REGEX',
 'MASTERCard_REGEX',
 'MD5_SHA_REGEX',
 'MOST_COMMON_PUNCT_REGEX',
 'NUMBERS_REGEX',
 'PHONE_REGEX',
 'PoBOX_REGEX',
 'SPECIAL_CHARACTERS_REGEX',
 'STOPWORDS',
 'STOPWORDS_de',
 'STOPWORDS_en',
 'STOPWORDS_es',
 'STOPWORDS_fr',
 'STOPWORDS_ru',
 'STOPWORDS_yo',
 'STREET_ADDRESS_REGEX',
 'TextFrame',
 'URL_PATTERN',
 'USER_HANDLES_REGEX',
 'VISACard_REGEX',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__generate_text',
 '__loader__',
 '__name__',
 '__numbers_dict',
 '__package__',
 '__spec__',
 '_lex_richness_herdan',
 '_lex_richness_maas_ttr',
 'clean_text',
 'defaultdict',
 'digit2words',
 'extract_btc_address',
 'extract_currencies',
 'extract_currency_symbols',
 'extract_dates',
 'extract_emails',
 'extract_emojis',
 'extract_hashtags',
 'extract_html_tags',
 'extract_mastercard_addr',
 'extract_md5sha',
 'extract_numbers',
 'extr

In [186]:
df['course_title'].iloc[1:5]

1    Complete GST Course & Certification - Grow You...
2    Financial Modeling for Business Analysts and C...
3    Beginner to Pro - Financial Analysis in Excel ...
4         How To Maximize Your Profits Trading Options
Name: course_title, dtype: object

In [187]:
# generating clean text by removing the stopwords and special characters
df['course_title']=df['course_title'].apply(nfx.remove_stopwords)
df['course_title']=df['course_title'].apply(nfx.remove_special_characters)
df['course_title'].iloc[1:5]

1    Complete GST Course  Certification  Grow Practice
2     Financial Modeling Business Analysts Consultants
3          Beginner Pro  Financial Analysis Excel 2017
4                     Maximize Profits Trading Options
Name: course_title, dtype: object

In [188]:
df.head()

course_title  \
0                 Ultimate Investment Banking Course   
1  Complete GST Course  Certification  Grow Practice   
2   Financial Modeling Business Analysts Consultants   
3        Beginner Pro  Financial Analysis Excel 2017   
4                   Maximize Profits Trading Options   

                                                 url  
0  https://www.udemy.com/ultimate-investment-bank...  
1      https://www.udemy.com/goods-and-services-tax/  
2  https://www.udemy.com/financial-modeling-for-b...  
3  https://www.udemy.com/complete-excel-finance-c...  
4  https://www.udemy.com/how-to-maximize-your-pro...

In [192]:
df=df.drop_duplicates('course_title')
df

course_title  \
0                    Ultimate Investment Banking Course   
1     Complete GST Course  Certification  Grow Practice   
2      Financial Modeling Business Analysts Consultants   
3           Beginner Pro  Financial Analysis Excel 2017   
4                      Maximize Profits Trading Options   
...                                                 ...   
3678    Learn jQuery Scratch  Master JavaScript library   
3679                    Design WordPress Website Coding   
3680                                Learn Build Polymer   
3681      CSS Animations Create Amazing Effects Website   
3682            MODX CMS Build Websites Beginners Guide   

                                                    url  
0     https://www.udemy.com/ultimate-investment-bank...  
1         https://www.udemy.com/goods-and-services-tax/  
2     https://www.udemy.com/financial-modeling-for-b...  
3     https://www.udemy.com/complete-excel-finance-c...  
4     https://www.udemy.com/how-to-maximize-your-pro...  
...                                                 ...  
3678  https://www.udemy.com/easy-jquery-for-beginner...  
3679  https://www.udemy.com/how-to-make-a-wordpress-...  
3680  https://www.udemy.com/learn-and-build-using-po...  
3681  https://www.udemy.com/css-animations-create-am...  
3682  https://www.udemy.com/using-modx-cms-to-build-...  

[3640 rows x 2 columns]

In [193]:
df.shape

(3640, 2)

In [194]:
python=df[df['course_title'].str.contains('Python')]
python.sample(2)

course_title  \
2965                             Projects Django Python   
3143  Complete Python Web Course Build 8 Python Web ...   

                                                    url  
2965  https://www.udemy.com/projects-in-django-and-p...  
3143  https://www.udemy.com/the-complete-python-web-...

In [195]:
df.dtypes

course_title    object
url             object
dtype: object

In [196]:
# vectorizing the course_title

countvect = CountVectorizer()

cv_mat = countvect.fit_transform(df['course_title'])

cv_mat

<3640x3564 sparse matrix of type '<class 'numpy.int64'>'
	with 18285 stored elements in Compressed Sparse Row format>

Here we are getting sparse matrix.

In [197]:
#cv_mat.todense() # .todense is the method to read sparse matrix
#dir(CountVectorizer)
#countvect.get_feature_names_out()
#df_cv_words = pd.DataFrame(cv_mat.todense(),columns=countvect.get_feature_names_out())

#df_cv_words

In [198]:
# cosine similarity matrix
cos=cosine_similarities = linear_kernel(cv_mat,cv_mat)

In [199]:
cos.shape

(3640, 3640)

## Recommendation

In [200]:
# drop duplicate
course_index=pd.Series(df.index,index=df['course_title']).drop_duplicates()
course_index

course_title
Ultimate Investment Banking Course                      0
Complete GST Course  Certification  Grow Practice       1
Financial Modeling Business Analysts Consultants        2
Beginner Pro  Financial Analysis Excel 2017             3
Maximize Profits Trading Options                        4
                                                     ... 
Learn jQuery Scratch  Master JavaScript library      3678
Design WordPress Website Coding                      3679
Learn Build Polymer                                  3680
CSS Animations Create Amazing Effects Website        3681
MODX CMS Build Websites Beginners Guide              3682
Length: 3640, dtype: int64

In [201]:
title1=course_index['Maximize Profits Trading Options']
title1

4

In [202]:
#list(enumerate(cos[title1])) # result index position with cosine value

In [203]:
#list(enumerate(cos[title1]))

In [204]:
def recommend_course(title):
    course_index=pd.Series(df.index, index=df['course_title']).drop_duplicates()
    
    index=course_index[title] # Get the index of the movie that matches the title
    
    sim=list(enumerate(cos[index])) # Get the pairwsie similarity scores of all movies with that movie
    
    sim_sorted=sorted(sim,key=lambda x: x[1],reverse=True) # Sort the movies based on the similarity scores
    
    sim_sorted_10=sim_sorted[1:10] # Get the scores of the 10 most similar course
    
    select_course_index=[i[0] for i in sim_sorted_10] #Get the course indices
    
    selected_course_score = [i[1] for i in sim_sorted_10] #Get the couser similarity score
    
    rec_df =data.iloc[select_course_index] # Return the top most similar couse
    
    rec_df['Similarity_Score'] = selected_course_score
    
    final_recommended_courses = rec_df[['course_title', 'Similarity_Score', 'url','price', 'num_subscribers']] #
    
    final_recommended_courses=final_recommended_courses.head(10)
    
    return final_recommended_courses
    

In [205]:
print("\n Recommended 10 Top course ")
ans = recommend_course('Trading Options Basics')
ans


 Recommended 10 Top course 


course_title  Similarity_Score  \
43    Options Trading - How to Win with Weekly Options               3.0   
59   How to Buy Cheap Options - Options Trading Pri...               3.0   
66            Options Trading Basics (3-Course Bundle)               3.0   
71   Options Trading Introduction: Day Trade Stock ...               3.0   
96   Intermediate Options trading concepts for Stoc...               3.0   
97                     Options Trading 101: The Basics               3.0   
138  Forex Trading with Fixed 'Risk through Options...               3.0   
399  Leaps Options Trading System - Diagonal Leaps ...               3.0   
410                             Trading Options Basics               3.0   

                                                   url  price  num_subscribers  
43   https://www.udemy.com/work-from-home-setup-you...    115             7489  
59   https://www.udemy.com/options-black-scholes-mo...    200              658  
66   https://www.udemy.com/learn-options-trading-co...    180            10100  
71   https://www.udemy.com/how-to-trade-stock-optio...     95            12394  
96   https://www.udemy.com/intermediate-options-tra...     40             2000  
97   https://www.udemy.com/options-trading-101-the-...      0             1514  
138                https://www.udemy.com/forexoptions/    200              611  
399  https://www.udemy.com/leaps-options-trading-sy...    200              993  
410      https://www.udemy.com/trading-options-basics/    200                8

In [206]:
print("\n Recommended 6 Top course ")
ans2 = recommend_course('Ultimate Investment Banking Course')
ans2


 Recommended 6 Top course 


course_title  Similarity_Score  \
39         The Complete Investment Banking Course 2017               3.0   
139  Intro to Investment Banking, M&A, IPO, Modelin...               2.0   
229  Investment Banking: How to Land a Job on Wall ...               2.0   
242         Advanced Accounting for Investment Banking               2.0   
419          The Investment Banking Recruitment Series               2.0   
457  Cryptocurrency (BTC & ETH) Investment & Tradin...               2.0   
640        Bitcoin & Ethereum Course (2 Course Bundle)               2.0   
659  Financial Accounting - The Ultimate Beginner C...               2.0   
721               Introduction to Financial Statements               2.0   

                                                   url  price  num_subscribers  
39   https://www.udemy.com/the-complete-investment-...    195             8575  
139  https://www.udemy.com/investment-banking-merge...     60             1922  
229  https://www.udemy.com/how-to-land-a-job-on-wal...     75             1218  
242  https://www.udemy.com/advanced-accounting-for-...     50             1260  
419           https://www.udemy.com/investmentbanking/     40               17  
457  https://www.udemy.com/cryptocurrency-btc-eth-i...     20                0  
640     https://www.udemy.com/bitcoin-ethereum-bundle/    200                1  
659  https://www.udemy.com/financial-accounting-the...    100                9  
721  https://www.udemy.com/introduction-to-financia...     40              112

In [207]:
df.head(2)

course_title  \
0                 Ultimate Investment Banking Course   
1  Complete GST Course  Certification  Grow Practice   

                                                 url  
0  https://www.udemy.com/ultimate-investment-bank...  
1      https://www.udemy.com/goods-and-services-tax/

In [208]:
import joblib
joblib.dump(df,open("data.joblib","wb"))

In [209]:
joblib.load(open("data.joblib","rb"))

course_title  \
0                    Ultimate Investment Banking Course   
1     Complete GST Course  Certification  Grow Practice   
2      Financial Modeling Business Analysts Consultants   
3           Beginner Pro  Financial Analysis Excel 2017   
4                      Maximize Profits Trading Options   
...                                                 ...   
3678    Learn jQuery Scratch  Master JavaScript library   
3679                    Design WordPress Website Coding   
3680                                Learn Build Polymer   
3681      CSS Animations Create Amazing Effects Website   
3682            MODX CMS Build Websites Beginners Guide   

                                                    url  
0     https://www.udemy.com/ultimate-investment-bank...  
1         https://www.udemy.com/goods-and-services-tax/  
2     https://www.udemy.com/financial-modeling-for-b...  
3     https://www.udemy.com/complete-excel-finance-c...  
4     https://www.udemy.com/how-to-maximize-your-pro...  
...                                                 ...  
3678  https://www.udemy.com/easy-jquery-for-beginner...  
3679  https://www.udemy.com/how-to-make-a-wordpress-...  
3680  https://www.udemy.com/learn-and-build-using-po...  
3681  https://www.udemy.com/css-animations-create-am...  
3682  https://www.udemy.com/using-modx-cms-to-build-...  

[3640 rows x 2 columns]

In [210]:
joblib.dump(cosine_similarities,open("cos.joblib","wb"))

In [211]:
user_input='Ultimate Investment Banking Course'

In [212]:
df.iloc[748]

course_title                    Forex  Equity Management Strategy
url             https://www.udemy.com/forex-millionaire-money-...
Name: 751, dtype: object

In [213]:
[(409, 6.0), (748, 6.0), (1168, 6.0), (592, 4.0), (726, 4.0)]

[(409, 6.0), (748, 6.0), (1168, 6.0), (592, 4.0), (726, 4.0)]

In [214]:
df.shape

(3640, 2)

### Code for app.py

In [261]:
user_input='Professional Risk Manager PRM Certification Level 3'
index=np.where(df['course_title']==user_input)[0][0]
similar_items=sorted(list(enumerate(cos[index])) ,key=lambda x: x[1],reverse=True) [1:10]
similar_items
data=[]
for i in similar_items:
    item=[]
    select_course_index =i[0]
    select_similarity_score=i[1]
    
    #print(select_course_index,select_similarity_score)
    recommend_course=df.iloc[i[0]]
    #print(recommend_course,select_course_index,select_similarity_score)
    recommend_course['select_course_index']=select_course_index
    recommend_course['select_similarity_score']=select_similarity_score
    #print(recommend_course)
    final_recommended_courses = recommend_course[['course_title','url']]
    #print(final_recommended_courses)
    final_recommended_courses=final_recommended_courses.head(10)
    item.append((final_recommended_courses['course_title']))
    item.append((final_recommended_courses['url']))
    #print(item)
    data.append(item)
    
print(data)

[['Professional Risk Manager PRM Certification Level 2', 'https://www.udemy.com/professional-risk-manager-prm-certification-level-2/'], ['Professional Risk Manager PRM Certification Level 3', 'https://www.udemy.com/professional-risk-manager-prm-certification-level-3/'], ['Professional Risk Manager PRM Certification Level 4', 'https://www.udemy.com/professional-risk-manager-prm-certification-level-4/'], ['Financial Risk Manager FRM Certification Level II', 'https://www.udemy.com/financial-risk-manager-frm-certification-level-ii/'], ['Financial Risk Manager FRM Certification Level', 'https://www.udemy.com/financial-risk-manager-frm-certification-level-i/'], ['Auditing Basics Professional Course Level', 'https://www.udemy.com/basics-of-auditing/'], ['Accounting Standards Basics Professional Course Level', 'https://www.udemy.com/basics-of-accounting-standards/'], ['Leasing  Comprehensive Study Professional Course Level', 'https://www.udemy.com/leasing-a-comprehensive-study/'], ['Accounting

In [33]:
joblib.dump(cosine_similarities,open("cos.joblib","wb"))

In [34]:
joblib.load(open("cos.joblib","rb"))

array([[4., 1., 0., ..., 0., 0., 0.],
       [1., 6., 0., ..., 0., 0., 0.],
       [0., 0., 5., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 3., 0., 1.],
       [0., 0., 0., ..., 0., 6., 0.],
       [0., 0., 0., ..., 1., 0., 6.]])

In [35]:
df

course_title  \
0                    Ultimate Investment Banking Course   
1     Complete GST Course  Certification  Grow Practice   
2      Financial Modeling Business Analysts Consultants   
3           Beginner Pro  Financial Analysis Excel 2017   
4                      Maximize Profits Trading Options   
...                                                 ...   
3678    Learn jQuery Scratch  Master JavaScript library   
3679                    Design WordPress Website Coding   
3680                                Learn Build Polymer   
3681      CSS Animations Create Amazing Effects Website   
3682            MODX CMS Build Websites Beginners Guide   

                                                    url  
0     https://www.udemy.com/ultimate-investment-bank...  
1         https://www.udemy.com/goods-and-services-tax/  
2     https://www.udemy.com/financial-modeling-for-b...  
3     https://www.udemy.com/complete-excel-finance-c...  
4     https://www.udemy.com/how-to-maximize-your-pro...  
...                                                 ...  
3678  https://www.udemy.com/easy-jquery-for-beginner...  
3679  https://www.udemy.com/how-to-make-a-wordpress-...  
3680  https://www.udemy.com/learn-and-build-using-po...  
3681  https://www.udemy.com/css-animations-create-am...  
3682  https://www.udemy.com/using-modx-cms-to-build-...  

[3683 rows x 2 columns]

In [36]:
course_title=list(df['course_title'].values)
course_title

joblib.dump(course_title,open("course_title.joblib","wb"))
joblib.load(open("course_title.joblib","rb"))



['Ultimate Investment Banking Course',
 'Complete GST Course  Certification  Grow Practice',
 'Financial Modeling Business Analysts Consultants',
 'Beginner Pro  Financial Analysis Excel 2017',
 'Maximize Profits Trading Options',
 'Trading Penny Stocks Guide Levels 2017',
 'Investing Trading Beginners Mastering Price Charts',
 'Trading Stock Chart Patterns Immediate Explosive Gains',
 'Options Trading 3  Advanced Stock Profit Success Method',
 'Investment Strategy Need Retirement',
 'Forex Trading Secrets Pros Amazons AWS',
 'Trading Options Money Flow',
 'Financial Management Risk Return Securities',
 'Forex Trading Course Work Smarter Harder Proven Results',
 'Python Algo Trading Sentiment Trading News',
 'Short Selling Learn Sell Stocks Fall',
 'Basic Technical Analysis Learn structure market',
 'Complete Chart Pattern Trading Course Proven Approach',
 '7 Deadly Mistakes Investing Slash Profits',
 'Financial Statements Easy',
 'Winning Forex Trading Live Forex Trading Examples',
 '

In [37]:

joblib.dump(df,open("data.joblib","wb"))
joblib.load(open("data.joblib","rb"))


course_title  \
0                    Ultimate Investment Banking Course   
1     Complete GST Course  Certification  Grow Practice   
2      Financial Modeling Business Analysts Consultants   
3           Beginner Pro  Financial Analysis Excel 2017   
4                      Maximize Profits Trading Options   
...                                                 ...   
3678    Learn jQuery Scratch  Master JavaScript library   
3679                    Design WordPress Website Coding   
3680                                Learn Build Polymer   
3681      CSS Animations Create Amazing Effects Website   
3682            MODX CMS Build Websites Beginners Guide   

                                                    url  
0     https://www.udemy.com/ultimate-investment-bank...  
1         https://www.udemy.com/goods-and-services-tax/  
2     https://www.udemy.com/financial-modeling-for-b...  
3     https://www.udemy.com/complete-excel-finance-c...  
4     https://www.udemy.com/how-to-maximize-your-pro...  
...                                                 ...  
3678  https://www.udemy.com/easy-jquery-for-beginner...  
3679  https://www.udemy.com/how-to-make-a-wordpress-...  
3680  https://www.udemy.com/learn-and-build-using-po...  
3681  https://www.udemy.com/css-animations-create-am...  
3682  https://www.udemy.com/using-modx-cms-to-build-...  

[3683 rows x 2 columns]

In [38]:
df

course_title  \
0                    Ultimate Investment Banking Course   
1     Complete GST Course  Certification  Grow Practice   
2      Financial Modeling Business Analysts Consultants   
3           Beginner Pro  Financial Analysis Excel 2017   
4                      Maximize Profits Trading Options   
...                                                 ...   
3678    Learn jQuery Scratch  Master JavaScript library   
3679                    Design WordPress Website Coding   
3680                                Learn Build Polymer   
3681      CSS Animations Create Amazing Effects Website   
3682            MODX CMS Build Websites Beginners Guide   

                                                    url  
0     https://www.udemy.com/ultimate-investment-bank...  
1         https://www.udemy.com/goods-and-services-tax/  
2     https://www.udemy.com/financial-modeling-for-b...  
3     https://www.udemy.com/complete-excel-finance-c...  
4     https://www.udemy.com/how-to-maximize-your-pro...  
...                                                 ...  
3678  https://www.udemy.com/easy-jquery-for-beginner...  
3679  https://www.udemy.com/how-to-make-a-wordpress-...  
3680  https://www.udemy.com/learn-and-build-using-po...  
3681  https://www.udemy.com/css-animations-create-am...  
3682  https://www.udemy.com/using-modx-cms-to-build-...  

[3683 rows x 2 columns]

In [39]:
course_title=list(df['course_title'].values)
course_title

['Ultimate Investment Banking Course',
 'Complete GST Course  Certification  Grow Practice',
 'Financial Modeling Business Analysts Consultants',
 'Beginner Pro  Financial Analysis Excel 2017',
 'Maximize Profits Trading Options',
 'Trading Penny Stocks Guide Levels 2017',
 'Investing Trading Beginners Mastering Price Charts',
 'Trading Stock Chart Patterns Immediate Explosive Gains',
 'Options Trading 3  Advanced Stock Profit Success Method',
 'Investment Strategy Need Retirement',
 'Forex Trading Secrets Pros Amazons AWS',
 'Trading Options Money Flow',
 'Financial Management Risk Return Securities',
 'Forex Trading Course Work Smarter Harder Proven Results',
 'Python Algo Trading Sentiment Trading News',
 'Short Selling Learn Sell Stocks Fall',
 'Basic Technical Analysis Learn structure market',
 'Complete Chart Pattern Trading Course Proven Approach',
 '7 Deadly Mistakes Investing Slash Profits',
 'Financial Statements Easy',
 'Winning Forex Trading Live Forex Trading Examples',
 '

In [40]:
dump=joblib.load(open("data.joblib","rb"))
dump

course_title  \
0                    Ultimate Investment Banking Course   
1     Complete GST Course  Certification  Grow Practice   
2      Financial Modeling Business Analysts Consultants   
3           Beginner Pro  Financial Analysis Excel 2017   
4                      Maximize Profits Trading Options   
...                                                 ...   
3678    Learn jQuery Scratch  Master JavaScript library   
3679                    Design WordPress Website Coding   
3680                                Learn Build Polymer   
3681      CSS Animations Create Amazing Effects Website   
3682            MODX CMS Build Websites Beginners Guide   

                                                    url  
0     https://www.udemy.com/ultimate-investment-bank...  
1         https://www.udemy.com/goods-and-services-tax/  
2     https://www.udemy.com/financial-modeling-for-b...  
3     https://www.udemy.com/complete-excel-finance-c...  
4     https://www.udemy.com/how-to-maximize-your-pro...  
...                                                 ...  
3678  https://www.udemy.com/easy-jquery-for-beginner...  
3679  https://www.udemy.com/how-to-make-a-wordpress-...  
3680  https://www.udemy.com/learn-and-build-using-po...  
3681  https://www.udemy.com/css-animations-create-am...  
3682  https://www.udemy.com/using-modx-cms-to-build-...  

[3683 rows x 2 columns]

In [41]:
dump.dtypes

course_title    object
url             object
dtype: object

In [259]:
dump.dtypes

course_title       object
url                object
price               int64
num_subscribers     int64
dtype: object

In [260]:
import pandas as pd
import json


df_dict = df.to_dict()
json_str = json.dumps(df_dict)
print(json_str)

{"course_title": {"0": "Ultimate Investment Banking Course", "1": "Complete GST Course  Certification  Grow Practice", "2": "Financial Modeling Business Analysts Consultants", "3": "Beginner Pro  Financial Analysis Excel 2017", "4": "Maximize Profits Trading Options", "5": "Trading Penny Stocks Guide Levels 2017", "6": "Investing Trading Beginners Mastering Price Charts", "7": "Trading Stock Chart Patterns Immediate Explosive Gains", "8": "Options Trading 3  Advanced Stock Profit Success Method", "9": "Investment Strategy Need Retirement", "10": "Forex Trading Secrets Pros Amazons AWS", "11": "Trading Options Money Flow", "12": "Financial Management Risk Return Securities", "13": "Forex Trading Course Work Smarter Harder Proven Results", "14": "Python Algo Trading Sentiment Trading News", "15": "Short Selling Learn Sell Stocks Fall", "16": "Basic Technical Analysis Learn structure market", "17": "Complete Chart Pattern Trading Course Proven Approach", "18": "7 Deadly Mistakes Investing

In [261]:
json_str

'{"course_title": {"0": "Ultimate Investment Banking Course", "1": "Complete GST Course  Certification  Grow Practice", "2": "Financial Modeling Business Analysts Consultants", "3": "Beginner Pro  Financial Analysis Excel 2017", "4": "Maximize Profits Trading Options", "5": "Trading Penny Stocks Guide Levels 2017", "6": "Investing Trading Beginners Mastering Price Charts", "7": "Trading Stock Chart Patterns Immediate Explosive Gains", "8": "Options Trading 3  Advanced Stock Profit Success Method", "9": "Investment Strategy Need Retirement", "10": "Forex Trading Secrets Pros Amazons AWS", "11": "Trading Options Money Flow", "12": "Financial Management Risk Return Securities", "13": "Forex Trading Course Work Smarter Harder Proven Results", "14": "Python Algo Trading Sentiment Trading News", "15": "Short Selling Learn Sell Stocks Fall", "16": "Basic Technical Analysis Learn structure market", "17": "Complete Chart Pattern Trading Course Proven Approach", "18": "7 Deadly Mistakes Investin

In [262]:
json_str.keys

AttributeError: 'str' object has no attribute 'keys'

In [263]:
import pandas as pd
import json
def serialize_dataframe(obj):
    if isinstance(obj, pd.DataFrame):
        return obj.to_dict()
    raise TypeError ("Type %s is not serializable" % type(obj))
 
json_str = json.dumps(df, default=serialize_dataframe)
print(json_str)

{"course_title": {"0": "Ultimate Investment Banking Course", "1": "Complete GST Course  Certification  Grow Practice", "2": "Financial Modeling Business Analysts Consultants", "3": "Beginner Pro  Financial Analysis Excel 2017", "4": "Maximize Profits Trading Options", "5": "Trading Penny Stocks Guide Levels 2017", "6": "Investing Trading Beginners Mastering Price Charts", "7": "Trading Stock Chart Patterns Immediate Explosive Gains", "8": "Options Trading 3  Advanced Stock Profit Success Method", "9": "Investment Strategy Need Retirement", "10": "Forex Trading Secrets Pros Amazons AWS", "11": "Trading Options Money Flow", "12": "Financial Management Risk Return Securities", "13": "Forex Trading Course Work Smarter Harder Proven Results", "14": "Python Algo Trading Sentiment Trading News", "15": "Short Selling Learn Sell Stocks Fall", "16": "Basic Technical Analysis Learn structure market", "17": "Complete Chart Pattern Trading Course Proven Approach", "18": "7 Deadly Mistakes Investing

In [265]:
dict_of_df = df.to_dict()

In [266]:
dict_of_df.keys()

dict_keys(['course_title', 'url', 'price', 'num_subscribers'])

In [271]:
dict_of_df['course_title']

{0: 'Ultimate Investment Banking Course',
 1: 'Complete GST Course  Certification  Grow Practice',
 2: 'Financial Modeling Business Analysts Consultants',
 3: 'Beginner Pro  Financial Analysis Excel 2017',
 4: 'Maximize Profits Trading Options',
 5: 'Trading Penny Stocks Guide Levels 2017',
 6: 'Investing Trading Beginners Mastering Price Charts',
 7: 'Trading Stock Chart Patterns Immediate Explosive Gains',
 8: 'Options Trading 3  Advanced Stock Profit Success Method',
 9: 'Investment Strategy Need Retirement',
 10: 'Forex Trading Secrets Pros Amazons AWS',
 11: 'Trading Options Money Flow',
 12: 'Financial Management Risk Return Securities',
 13: 'Forex Trading Course Work Smarter Harder Proven Results',
 14: 'Python Algo Trading Sentiment Trading News',
 15: 'Short Selling Learn Sell Stocks Fall',
 16: 'Basic Technical Analysis Learn structure market',
 17: 'Complete Chart Pattern Trading Course Proven Approach',
 18: '7 Deadly Mistakes Investing Slash Profits',
 19: 'Financial Stat

In [180]:
dict_of_df['course_title'][0]

'Ultimate Investment Banking Course'

In [272]:
ans.columns

Index(['course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers'], dtype='object')